In [3]:
pip install seaborn

In [6]:
pip install ping3

  Obtaining dependency information for ping3 from https://files.pythonhosted.org/packages/34/03/2580e349f6cab60865456db4b8984a50c91c98e88fc5055a3cd7c30485a5/ping3-5.1.5-py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install requests pandas schedule python-dotenv

  Obtaining dependency information for schedule from https://files.pythonhosted.org/packages/20/a7/84c96b61fd13205f2cafbe263cdb2745965974bdf3e0078f121dfeca5f02/schedule-1.2.2-py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [8]:
import requests
import time
import pandas as pd
import json
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime

class UltraStrictCryptoExchangeAnalyzer:
    def __init__(self):
        self.data_points = []
        self.platform_names = ['Binance', 'MEXC', 'Bitget']
    
    def get_binance_price_with_timestamp(self):
        """獲取帶時間戳的Binance價格"""
        request_time = time.time()
        try:
            resp = requests.get("https://api.binance.com/api/v3/ticker/24hr?symbol=BTCUSDT", timeout=15)
            data = resp.json()
            
            return {
                'success': True,
                'price': float(data['lastPrice']),
                'platform': 'Binance',
                'request_timestamp': request_time,
                'data_timestamp': data['closeTime'] / 1000,  # 毫秒轉秒
            }
        except Exception as e:
            return {
                'success': False,
                'platform': 'Binance',
                'request_timestamp': request_time,
                'error': str(e)
            }

    def get_mexc_price_with_timestamp(self):
        """獲取帶時間戳的MEXC價格"""
        request_time = time.time()
        try:
            resp = requests.get("https://api.mexc.com/api/v3/ticker/24hr?symbol=BTCUSDT", timeout=15)
            data = resp.json()
            
            return {
                'success': True,
                'price': float(data['lastPrice']),
                'platform': 'MEXC',
                'request_timestamp': request_time,
                'data_timestamp': data['closeTime'] / 1000,  # 毫秒轉秒
            }
        except Exception as e:
            return {
                'success': False,
                'platform': 'MEXC',
                'request_timestamp': request_time,
                'error': str(e)
            }

    def get_bitget_price_with_timestamp(self):
        """獲取帶時間戳的Bitget價格"""
        request_time = time.time()
        try:
            # Bitget v2 API endpoint
            resp = requests.get("https://api.bitget.com/api/v2/spot/market/ticker?symbol=BTCUSDT", timeout=15)
            data = resp.json()
            
            if data.get('code') == '00000' and data.get('data'):
                ticker_data = data['data'][0] if isinstance(data['data'], list) else data['data']
                return {
                    'success': True,
                    'price': float(ticker_data['lastPr']),
                    'platform': 'Bitget',
                    'request_timestamp': request_time,
                    'data_timestamp': int(ticker_data['ts']) / 1000,  # 毫秒轉秒
                }
            else:
                # 如果v2失敗，嘗試舊的API格式
                resp = requests.get("https://api.bitget.com/api/spot/v1/market/ticker?symbol=BTCUSDT_SPBL", timeout=15)
                data = resp.json()
                
                if data.get('code') == '00000' and data.get('data'):
                    ticker_data = data['data']
                    return {
                        'success': True,
                        'price': float(ticker_data['close']),
                        'platform': 'Bitget',
                        'request_timestamp': request_time,
                        'data_timestamp': int(ticker_data['ts']) / 1000,
                    }
                else:
                    raise Exception(f"Bitget API error: {data}")
                
        except Exception as e:
            return {
                'success': False,
                'platform': 'Bitget',
                'request_timestamp': request_time,
                'error': str(e)
            }

    def collect_single_data_point(self, session_name=""):
        """收集單次數據點"""
        timestamp = datetime.now()
        
        # 並發執行三個API調用
        with ThreadPoolExecutor(max_workers=3) as executor:
            future_binance = executor.submit(self.get_binance_price_with_timestamp)
            future_mexc = executor.submit(self.get_mexc_price_with_timestamp)
            future_bitget = executor.submit(self.get_bitget_price_with_timestamp)
            
            results = {
                'Binance': future_binance.result(),
                'MEXC': future_mexc.result(),
                'Bitget': future_bitget.result()
            }
        
        # 建立數據點記錄
        data_point = {
            'timestamp': timestamp.isoformat(),
            'session': session_name
        }
        
        # 記錄各平台數據
        successful_platforms = 0
        for platform_name, result in results.items():
            data_point[f'{platform_name.lower()}_success'] = result['success']
            if result['success']:
                data_point[f'{platform_name.lower()}_price'] = result['price']
                data_point[f'{platform_name.lower()}_data_timestamp'] = result['data_timestamp']
                data_point[f'{platform_name.lower()}_request_timestamp'] = result['request_timestamp']
                successful_platforms += 1
            else:
                print(f"  警告：{platform_name} API失敗 - {result.get('error', 'Unknown error')}")
        
        data_point['successful_platforms'] = successful_platforms
        
        # 輸出即時結果
        success_info = []
        for platform, result in results.items():
            if result['success']:
                success_info.append(f"{platform}: ${result['price']:,.2f}")
        
        if successful_platforms >= 2:
            print(f"[{data_point['timestamp'][:19]}] {' | '.join(success_info)} | 成功平台: {successful_platforms}/3")
        else:
            print(f"[{data_point['timestamp'][:19]}] 數據收集失敗，成功平台: {successful_platforms}/3")
        
        return data_point

    def collect_continuous_data(self, duration_minutes, interval_seconds=60, session_name="Ultra Strict Analysis"):
        """連續收集數據"""
        print(f"\n開始收集交易所數據比較")
        print(f"交易所: Binance、MEXC、Bitget")
        print(f"持續時間: {duration_minutes} 分鐘，間隔: {interval_seconds} 秒")
        print(f"目標數據量: 約 {int(duration_minutes * 60 / interval_seconds)} 筆")
        print(f"數據新鮮度標準: ≤0.2秒:100分 | 0.2-0.5秒:95-100分 | 0.5-1秒:80-95分 | >1秒:<80分")
        print("-" * 90)
        
        start_time = time.time()
        end_time = start_time + (duration_minutes * 60)
        collected_count = 0
        
        try:
            while time.time() < end_time:
                data_point = self.collect_single_data_point(session_name)
                self.data_points.append(data_point)
                collected_count += 1
                
                remaining_seconds = int(end_time - time.time())
                if remaining_seconds > interval_seconds:
                    print(f"已收集: {collected_count} 筆，剩餘時間: {remaining_seconds//60}分{remaining_seconds%60}秒，等待{interval_seconds}秒...")
                    time.sleep(interval_seconds)
                else:
                    break
                    
        except KeyboardInterrupt:
            print(f"\n收集被用戶中斷")
        
        print(f"\n數據收集完成，共收集 {collected_count} 筆數據")

    def calculate_consistency_scores(self):
        """計算各交易所一致性分數（嚴格標準）"""
        platform_consistency = {}
        
        for platform in self.platform_names:
            consistency_scores = []
            
            for data_point in self.data_points:
                platform_key = platform.lower()
                if not data_point.get(f'{platform_key}_success', False):
                    continue
                
                platform_price = data_point.get(f'{platform_key}_price')
                if platform_price is None:
                    continue
                
                # 獲取其他成功平台的價格
                other_prices = []
                for other_platform in self.platform_names:
                    if other_platform == platform:
                        continue
                    other_key = other_platform.lower()
                    if data_point.get(f'{other_key}_success', False):
                        other_price = data_point.get(f'{other_key}_price')
                        if other_price is not None:
                            other_prices.append(other_price)
                
                if len(other_prices) > 0:
                    # 計算與其他交易所的相對差異
                    avg_other_price = sum(other_prices) / len(other_prices)
                    relative_diff = abs(platform_price - avg_other_price) / avg_other_price
                    
                    # 嚴格的一致性分數：CI = max(0, 100 - relative_diff × 50000)
                    # 0.001% 差異扣 0.5分，0.01% 差異扣 5分，0.1% 差異扣 50分
                    consistency_score = max(0, 100 - relative_diff * 50000)
                    consistency_scores.append(consistency_score)
            
            if consistency_scores:
                platform_consistency[platform] = {
                    'individual_scores': consistency_scores,
                    'average_score': sum(consistency_scores) / len(consistency_scores),
                    'count': len(consistency_scores)
                }
            else:
                platform_consistency[platform] = {
                    'individual_scores': [],
                    'average_score': 0,
                    'count': 0
                }
        
        return platform_consistency

    def calculate_freshness_scores(self):
        """計算各交易所資料新鮮度分數（修改版：0.2秒門檻）"""
        platform_freshness = {}
        
        for platform in self.platform_names:
            freshness_scores = []
            
            for data_point in self.data_points:
                platform_key = platform.lower()
                if not data_point.get(f'{platform_key}_success', False):
                    continue
                
                data_timestamp = data_point.get(f'{platform_key}_data_timestamp')
                request_timestamp = data_point.get(f'{platform_key}_request_timestamp')
                
                if data_timestamp is not None and request_timestamp is not None:
                    # 計算資料延遲（秒）
                    data_delay = abs(request_timestamp - data_timestamp)  # 使用絕對值
                    
                    # 修改版資料新鮮度分數計算（0.2秒門檻）
                    if data_delay <= 0.2:      # ≤0.2秒：100分（準實時）
                        freshness_score = 100
                    elif data_delay <= 0.5:    # 0.2-0.5秒：100→95分（近實時）
                        freshness_score = 100 - (data_delay - 0.2) * 16.67  # 每0.1秒扣約1.67分
                    elif data_delay <= 1.0:    # 0.5-1秒：95→80分（可接受）
                        freshness_score = 95 - (data_delay - 0.5) * 30      # 每0.1秒扣3分
                    elif data_delay <= 2.0:    # 1-2秒：80→50分（滯後）
                        freshness_score = 80 - (data_delay - 1.0) * 30      # 每0.1秒扣3分
                    elif data_delay <= 5.0:    # 2-5秒：50→20分（嚴重滯後）
                        freshness_score = 50 - (data_delay - 2.0) * 10      # 每0.1秒扣1分
                    else:                      # >5秒：大幅扣分（不可接受）
                        freshness_score = max(0, 20 - (data_delay - 5.0) * 8)  # 每0.1秒扣0.8分
                    
                    freshness_scores.append({
                        'score': freshness_score,
                        'delay': data_delay
                    })
            
            if freshness_scores:
                avg_score = sum(item['score'] for item in freshness_scores) / len(freshness_scores)
                avg_delay = sum(item['delay'] for item in freshness_scores) / len(freshness_scores)
                
                platform_freshness[platform] = {
                    'individual_scores': freshness_scores,
                    'average_score': avg_score,
                    'average_delay': avg_delay,
                    'count': len(freshness_scores)
                }
            else:
                platform_freshness[platform] = {
                    'individual_scores': [],
                    'average_score': 0,
                    'average_delay': 0,
                    'count': 0
                }
        
        return platform_freshness

    def calculate_availability_scores(self):
        """計算各交易所可用性分數"""
        platform_availability = {}
        total_attempts = len(self.data_points)
        
        for platform in self.platform_names:
            platform_key = platform.lower()
            successful_calls = sum(1 for dp in self.data_points 
                                 if dp.get(f'{platform_key}_success', False))
            
            # 可用性分數 = 成功率 × 100
            availability_score = (successful_calls / total_attempts) * 100 if total_attempts > 0 else 0
            
            platform_availability[platform] = {
                'successful_calls': successful_calls,
                'total_calls': total_attempts,
                'availability_score': availability_score,
                'missing_rate': 100 - availability_score
            }
        
        return platform_availability

    def calculate_composite_dqi(self, weights={'consistency': 0.4, 'freshness': 0.4, 'availability': 0.2}):
        """計算各交易所綜合數據品質指數"""
        consistency_scores = self.calculate_consistency_scores()
        freshness_scores = self.calculate_freshness_scores()
        availability_scores = self.calculate_availability_scores()
        
        platform_dqi = {}
        
        for platform in self.platform_names:
            ci = consistency_scores[platform]['average_score']
            fi = freshness_scores[platform]['average_score']  
            ai = availability_scores[platform]['availability_score']
            
            # 綜合DQI計算: DQI = w1×CI + w2×FI + w3×AI
            dqi = (ci * weights['consistency'] + 
                   fi * weights['freshness'] + 
                   ai * weights['availability'])
            
            platform_dqi[platform] = {
                'consistency_index': ci,
                'freshness_index': fi,
                'availability_index': ai,
                'composite_dqi': dqi,
                'consistency_count': consistency_scores[platform]['count'],
                'freshness_count': freshness_scores[platform]['count'],
                'availability_stats': availability_scores[platform],
                'avg_delay_seconds': freshness_scores[platform]['average_delay']
            }
        
        return platform_dqi

    def print_detailed_analysis(self):
        """輸出詳細分析報告"""
        if not self.data_points:
            print("無數據可分析")
            return
        
        dqi_results = self.calculate_composite_dqi()
        
        print("\n" + "=" * 110)
        print("BTC三大交易所數據品質分析報告（修改版：0.2秒新鮮度門檻）")
        print("交易所: Binance、MEXC、Bitget")
        print("數據新鮮度標準: ≤0.2秒:100分 | 0.2-0.5秒:95-100分 | 0.5-1秒:80-95分 | >1秒:<80分")
        print("一致性評分標準: 0.001%差異扣0.5分 | 0.01%差異扣5分 | 0.1%差異扣50分")
        print("=" * 110)
        
        print(f"\n基本統計資訊:")
        print(f"總數據收集次數: {len(self.data_points)}")
        
        # 各交易所詳細分數
        print(f"\n各交易所詳細評分:")
        print(f"{'交易所':<15} {'一致性':<10} {'新鮮度':<10} {'平均延遲':<12} {'可用性':<10} {'綜合DQI':<10}")
        print("-" * 80)
        
        sorted_platforms = sorted(dqi_results.items(), 
                                key=lambda x: x[1]['composite_dqi'], 
                                reverse=True)
        
        for platform, scores in sorted_platforms:
            avg_delay = scores.get('avg_delay_seconds', 0)
            print(f"{platform:<15} "
                  f"{scores['consistency_index']:<10.2f} "
                  f"{scores['freshness_index']:<10.2f} "
                  f"{avg_delay:<12.3f}s "
                  f"{scores['availability_index']:<10.1f}% "
                  f"{scores['composite_dqi']:<10.2f}")
        
        print(f"\n交易所排名（依綜合DQI）:")
        for i, (platform, scores) in enumerate(sorted_platforms, 1):
            quality_level = self.get_quality_level(scores['composite_dqi'])
            print(f"{i}. {platform}: {scores['composite_dqi']:.2f}分 ({quality_level})")
        
        # 分數分布統計
        dqi_scores = [scores['composite_dqi'] for _, scores in dqi_results.items()]
        if len(dqi_scores) > 1:
            score_range = max(dqi_scores) - min(dqi_scores)
            print(f"\n評分差異統計:")
            print(f"最高分: {max(dqi_scores):.2f}")
            print(f"最低分: {min(dqi_scores):.2f}")
            print(f"分數範圍: {score_range:.2f}")
            print(f"平均分: {sum(dqi_scores)/len(dqi_scores):.2f}")
        
        # 新鮮度延遲詳細統計
        freshness_data = self.calculate_freshness_scores()
        print(f"\n數據新鮮度延遲詳細統計:")
        for platform in self.platform_names:
            if freshness_data[platform]['count'] > 0:
                delays = [item['delay'] for item in freshness_data[platform]['individual_scores']]
                min_delay = min(delays)
                max_delay = max(delays)
                print(f"{platform}: 延遲範圍 {min_delay:.3f}s - {max_delay:.3f}s | "
                      f"平均 {freshness_data[platform]['average_delay']:.3f}s | "
                      f"新鮮度分數 {freshness_data[platform]['average_score']:.1f}")
        
        # 詳細指標解析
        print(f"\n詳細指標解析:")
        for platform, scores in dqi_results.items():
            print(f"\n{platform}:")
            print(f"  一致性指標: {scores['consistency_index']:.2f}/100 "
                  f"(基於 {scores['consistency_count']} 次有效比較)")
            print(f"  新鮮度指標: {scores['freshness_index']:.2f}/100 "
                  f"(基於 {scores['freshness_count']} 次有效測量，平均延遲 {scores['avg_delay_seconds']:.3f}秒)")
            print(f"  可用性指標: {scores['availability_index']:.1f}% "
                  f"({scores['availability_stats']['successful_calls']}/{scores['availability_stats']['total_calls']})")

    def get_quality_level(self, dqi_score):
        """根據DQI分數返回品質等級（嚴格標準）"""
        if dqi_score >= 95:
            return "優秀"
        elif dqi_score >= 85:
            return "良好"
        elif dqi_score >= 70:
            return "普通"
        elif dqi_score >= 50:
            return "待改善"
        else:
            return "需改善"

    def save_analysis_results(self, filename_prefix="btc_exchange_modified_analysis"):
        """保存分析結果"""
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        
        # 保存原始數據
        df = pd.DataFrame(self.data_points)
        raw_filename = f"{filename_prefix}_raw_{timestamp}.csv"
        df.to_csv(raw_filename, index=False, encoding='utf-8-sig')
        
        # 保存分析結果
        dqi_results = self.calculate_composite_dqi()
        analysis_filename = f"{filename_prefix}_dqi_{timestamp}.json"
        
        with open(analysis_filename, 'w', encoding='utf-8') as f:
            json.dump(dqi_results, f, ensure_ascii=False, indent=2)
        
        print(f"\n分析結果已保存:")
        print(f"原始數據: {raw_filename}")
        print(f"DQI分析: {analysis_filename}")
        
        return raw_filename, analysis_filename

def main():
    analyzer = UltraStrictCryptoExchangeAnalyzer()
    
    print("BTC三大交易所數據品質分析系統")
    print("交易所: Binance、MEXC、Bitget")
    print("=" * 80)
    
    choice = input("選擇執行模式:\n1=單次測試\n2=連續收集\n請選擇: ").strip()
    
    if choice == "1":
        # 單次測試
        data_point = analyzer.collect_single_data_point("Modified Strict Test")
        analyzer.data_points.append(data_point)
        
        if data_point['successful_platforms'] >= 2:
            analyzer.print_detailed_analysis()
    
    elif choice == "2":
        # 連續收集
        try:
            duration = float(input("請輸入收集時間(分鐘，建議480分鐘=8小時): ").strip())
            interval = float(input("請輸入收集間隔(秒，建議60秒): ").strip())
            
            if duration <= 0 or interval <= 0:
                print("時間與間隔必須大於0")
            else:
                # 計算預期數據量
                expected_count = int(duration * 60 / interval)
                print(f"預期收集 {expected_count} 筆數據")
                confirm = input(f"確認開始收集？這將持續 {duration/60:.1f} 小時 (y/N): ").strip().lower()
                if confirm == 'y':
                    analyzer.collect_continuous_data(duration, interval, "Modified Strict Full Collection")
                    analyzer.print_detailed_analysis()
                    analyzer.save_analysis_results()
        except ValueError:
            print("請輸入有效數字")
    
    else:
        print("無效選擇，執行單次測試")
        data_point = analyzer.collect_single_data_point("Modified Strict Default Test")
        analyzer.data_points.append(data_point)
        analyzer.print_detailed_analysis()

if __name__ == "__main__":
    main()

BTC三大交易所數據品質分析系統
交易所: Binance、MEXC、Bitget
選擇執行模式:
1=單次測試
2=連續收集
請選擇: 2
請輸入收集時間(分鐘，建議480分鐘=8小時): 480
請輸入收集間隔(秒，建議60秒): 60
預期收集 480 筆數據
確認開始收集？這將持續 8.0 小時 (y/N): y

開始收集交易所數據比較
交易所: Binance、MEXC、Bitget
持續時間: 480.0 分鐘，間隔: 60.0 秒
目標數據量: 約 480 筆
數據新鮮度標準: ≤0.2秒:100分 | 0.2-0.5秒:95-100分 | 0.5-1秒:80-95分 | >1秒:<80分
------------------------------------------------------------------------------------------
[2025-09-06T11:15:39] Binance: $111,120.26 | MEXC: $111,121.08 | Bitget: $111,128.43 | 成功平台: 3/3
已收集: 1 筆，剩餘時間: 479分57秒，等待60.0秒...
[2025-09-06T11:16:41] Binance: $111,145.37 | MEXC: $111,146.16 | Bitget: $111,148.90 | 成功平台: 3/3
已收集: 2 筆，剩餘時間: 478分55秒，等待60.0秒...
[2025-09-06T11:17:43] Binance: $111,145.37 | MEXC: $111,146.05 | Bitget: $111,150.31 | 成功平台: 3/3
已收集: 3 筆，剩餘時間: 477分53秒，等待60.0秒...
[2025-09-06T11:18:45] Binance: $111,171.15 | MEXC: $111,171.79 | Bitget: $111,183.59 | 成功平台: 3/3
已收集: 4 筆，剩餘時間: 476分51秒，等待60.0秒...
[2025-09-06T11:19:47] Binance: $111,138.66 | MEXC: $111,139.31 | Bitget: $111,

[2025-09-06T12:16:49] Binance: $110,863.33 | MEXC: $110,865.01 | Bitget: $110,873.00 | 成功平台: 3/3
已收集: 60 筆，剩餘時間: 418分47秒，等待60.0秒...
[2025-09-06T12:17:51] Binance: $110,859.24 | MEXC: $110,860.62 | Bitget: $110,867.94 | 成功平台: 3/3
已收集: 61 筆，剩餘時間: 417分45秒，等待60.0秒...
[2025-09-06T12:18:53] Binance: $110,857.38 | MEXC: $110,858.75 | Bitget: $110,862.10 | 成功平台: 3/3
已收集: 62 筆，剩餘時間: 416分43秒，等待60.0秒...
[2025-09-06T12:19:55] Binance: $110,848.00 | MEXC: $110,849.37 | Bitget: $110,862.09 | 成功平台: 3/3
已收集: 63 筆，剩餘時間: 415分40秒，等待60.0秒...
[2025-09-06T12:20:58] Binance: $110,847.99 | MEXC: $110,849.37 | Bitget: $110,862.10 | 成功平台: 3/3
已收集: 64 筆，剩餘時間: 414分38秒，等待60.0秒...
[2025-09-06T12:22:00] Binance: $110,811.27 | MEXC: $110,790.91 | Bitget: $110,812.00 | 成功平台: 3/3
已收集: 65 筆，剩餘時間: 413分36秒，等待60.0秒...
[2025-09-06T12:23:02] Binance: $110,816.44 | MEXC: $110,817.35 | Bitget: $110,824.92 | 成功平台: 3/3
已收集: 66 筆，剩餘時間: 412分34秒，等待60.0秒...
[2025-09-06T12:24:04] Binance: $110,811.22 | MEXC: $110,812.08 | Bitget: $11

[2025-09-06T13:21:06] Binance: $110,805.33 | MEXC: $110,803.72 | Bitget: $110,810.72 | 成功平台: 3/3
已收集: 122 筆，剩餘時間: 354分30秒，等待60.0秒...
[2025-09-06T13:22:08] Binance: $110,796.34 | MEXC: $110,803.71 | Bitget: $110,799.00 | 成功平台: 3/3
已收集: 123 筆，剩餘時間: 353分28秒，等待60.0秒...
[2025-09-06T13:23:10] Binance: $110,776.94 | MEXC: $110,786.29 | Bitget: $110,786.18 | 成功平台: 3/3
已收集: 124 筆，剩餘時間: 352分26秒，等待60.0秒...
[2025-09-06T13:24:12] Binance: $110,733.11 | MEXC: $110,731.59 | Bitget: $110,731.01 | 成功平台: 3/3
已收集: 125 筆，剩餘時間: 351分24秒，等待60.0秒...
[2025-09-06T13:25:14] Binance: $110,705.51 | MEXC: $110,711.71 | Bitget: $110,710.15 | 成功平台: 3/3
已收集: 126 筆，剩餘時間: 350分22秒，等待60.0秒...
[2025-09-06T13:26:16] Binance: $110,764.35 | MEXC: $110,752.60 | Bitget: $110,769.96 | 成功平台: 3/3
已收集: 127 筆，剩餘時間: 349分19秒，等待60.0秒...
[2025-09-06T13:27:19] Binance: $110,721.05 | MEXC: $110,719.25 | Bitget: $110,728.45 | 成功平台: 3/3
已收集: 128 筆，剩餘時間: 348分17秒，等待60.0秒...
[2025-09-06T13:28:21] Binance: $110,751.07 | MEXC: $110,735.71 | Bitg

[2025-09-06T14:25:24] Binance: $110,716.00 | MEXC: $110,711.61 | Bitget: $110,728.00 | 成功平台: 3/3
已收集: 184 筆，剩餘時間: 290分12秒，等待60.0秒...
[2025-09-06T14:26:26] Binance: $110,716.01 | MEXC: $110,711.60 | Bitget: $110,735.78 | 成功平台: 3/3
已收集: 185 筆，剩餘時間: 289分10秒，等待60.0秒...
[2025-09-06T14:27:28] Binance: $110,770.52 | MEXC: $110,765.99 | Bitget: $110,781.66 | 成功平台: 3/3
已收集: 186 筆，剩餘時間: 288分7秒，等待60.0秒...
[2025-09-06T14:28:31] Binance: $110,787.12 | MEXC: $110,782.63 | Bitget: $110,795.75 | 成功平台: 3/3
已收集: 187 筆，剩餘時間: 287分5秒，等待60.0秒...
[2025-09-06T14:29:33] Binance: $110,787.12 | MEXC: $110,782.92 | Bitget: $110,795.76 | 成功平台: 3/3
已收集: 188 筆，剩餘時間: 286分3秒，等待60.0秒...
[2025-09-06T14:30:35] Binance: $110,787.12 | MEXC: $110,782.84 | Bitget: $110,795.76 | 成功平台: 3/3
已收集: 189 筆，剩餘時間: 285分1秒，等待60.0秒...
[2025-09-06T14:31:37] Binance: $110,817.11 | MEXC: $110,782.83 | Bitget: $110,824.26 | 成功平台: 3/3
已收集: 190 筆，剩餘時間: 283分58秒，等待60.0秒...
[2025-09-06T14:32:40] Binance: $110,786.08 | MEXC: $110,781.82 | Bitget: 

[2025-09-06T15:29:42] Binance: $110,817.00 | MEXC: $110,811.95 | Bitget: $110,832.72 | 成功平台: 3/3
已收集: 246 筆，剩餘時間: 225分54秒，等待60.0秒...
[2025-09-06T15:30:44] Binance: $110,817.00 | MEXC: $110,811.95 | Bitget: $110,832.71 | 成功平台: 3/3
已收集: 247 筆，剩餘時間: 224分52秒，等待60.0秒...
[2025-09-06T15:31:46] Binance: $110,817.00 | MEXC: $110,811.96 | Bitget: $110,832.72 | 成功平台: 3/3
已收集: 248 筆，剩餘時間: 223分50秒，等待60.0秒...
[2025-09-06T15:32:48] Binance: $110,816.99 | MEXC: $110,811.96 | Bitget: $110,832.72 | 成功平台: 3/3
已收集: 249 筆，剩餘時間: 222分47秒，等待60.0秒...
[2025-09-06T15:33:51] Binance: $110,816.99 | MEXC: $110,811.88 | Bitget: $110,832.71 | 成功平台: 3/3
已收集: 250 筆，剩餘時間: 221分45秒，等待60.0秒...
[2025-09-06T15:34:53] Binance: $110,816.99 | MEXC: $110,811.68 | Bitget: $110,832.71 | 成功平台: 3/3
已收集: 251 筆，剩餘時間: 220分43秒，等待60.0秒...
[2025-09-06T15:35:55] Binance: $110,817.00 | MEXC: $110,805.94 | Bitget: $110,832.71 | 成功平台: 3/3
已收集: 252 筆，剩餘時間: 219分41秒，等待60.0秒...
[2025-09-06T15:36:57] Binance: $110,803.01 | MEXC: $110,805.93 | Bitg

[2025-09-06T16:34:00] Binance: $110,829.99 | MEXC: $110,819.13 | Bitget: $110,855.69 | 成功平台: 3/3
已收集: 308 筆，剩餘時間: 161分36秒，等待60.0秒...
[2025-09-06T16:35:02] Binance: $110,864.28 | MEXC: $110,853.72 | Bitget: $110,879.87 | 成功平台: 3/3
已收集: 309 筆，剩餘時間: 160分33秒，等待60.0秒...
[2025-09-06T16:36:05] Binance: $110,864.28 | MEXC: $110,853.68 | Bitget: $110,879.86 | 成功平台: 3/3
已收集: 310 筆，剩餘時間: 159分31秒，等待60.0秒...
[2025-09-06T16:37:07] Binance: $110,864.29 | MEXC: $110,853.68 | Bitget: $110,885.27 | 成功平台: 3/3
已收集: 311 筆，剩餘時間: 158分29秒，等待60.0秒...
[2025-09-06T16:38:09] Binance: $110,864.29 | MEXC: $110,853.68 | Bitget: $110,885.28 | 成功平台: 3/3
已收集: 312 筆，剩餘時間: 157分26秒，等待60.0秒...
[2025-09-06T16:39:12] Binance: $110,864.29 | MEXC: $110,853.73 | Bitget: $110,885.27 | 成功平台: 3/3
已收集: 313 筆，剩餘時間: 156分24秒，等待60.0秒...
[2025-09-06T16:40:14] Binance: $110,864.29 | MEXC: $110,853.73 | Bitget: $110,879.97 | 成功平台: 3/3
已收集: 314 筆，剩餘時間: 155分22秒，等待60.0秒...
[2025-09-06T16:41:16] Binance: $110,864.28 | MEXC: $110,853.76 | Bitg

[2025-09-06T17:38:20] Binance: $110,802.59 | MEXC: $110,793.19 | Bitget: $110,811.05 | 成功平台: 3/3
已收集: 370 筆，剩餘時間: 97分15秒，等待60.0秒...
[2025-09-06T17:39:23] Binance: $110,793.99 | MEXC: $110,788.08 | Bitget: $110,805.44 | 成功平台: 3/3
已收集: 371 筆，剩餘時間: 96分13秒，等待60.0秒...
[2025-09-06T17:40:25] Binance: $110,781.67 | MEXC: $110,775.74 | Bitget: $110,794.37 | 成功平台: 3/3
已收集: 372 筆，剩餘時間: 95分11秒，等待60.0秒...
[2025-09-06T17:41:27] Binance: $110,810.68 | MEXC: $110,787.49 | Bitget: $110,814.07 | 成功平台: 3/3
已收集: 373 筆，剩餘時間: 94分9秒，等待60.0秒...
[2025-09-06T17:42:29] Binance: $110,810.69 | MEXC: $110,805.55 | Bitget: $110,814.08 | 成功平台: 3/3
已收集: 374 筆，剩餘時間: 93分7秒，等待60.0秒...
[2025-09-06T17:43:31] Binance: $110,810.69 | MEXC: $110,805.46 | Bitget: $110,814.08 | 成功平台: 3/3
已收集: 375 筆，剩餘時間: 92分4秒，等待60.0秒...
[2025-09-06T17:44:34] Binance: $110,810.69 | MEXC: $110,805.46 | Bitget: $110,814.07 | 成功平台: 3/3
已收集: 376 筆，剩餘時間: 91分2秒，等待60.0秒...
[2025-09-06T17:45:36] Binance: $110,817.11 | MEXC: $110,807.30 | Bitget: $110,82

[2025-09-06T18:43:41] Binance: $110,758.77 | MEXC: $110,756.32 | Bitget: $110,761.51 | 成功平台: 3/3
已收集: 433 筆，剩餘時間: 31分55秒，等待60.0秒...
[2025-09-06T18:44:43] Binance: $110,758.77 | MEXC: $110,756.31 | Bitget: $110,761.51 | 成功平台: 3/3
已收集: 434 筆，剩餘時間: 30分53秒，等待60.0秒...
[2025-09-06T18:45:45] Binance: $110,758.77 | MEXC: $110,756.29 | Bitget: $110,761.50 | 成功平台: 3/3
已收集: 435 筆，剩餘時間: 29分50秒，等待60.0秒...
[2025-09-06T18:46:48] Binance: $110,758.77 | MEXC: $110,756.24 | Bitget: $110,761.51 | 成功平台: 3/3
已收集: 436 筆，剩餘時間: 28分48秒，等待60.0秒...
[2025-09-06T18:47:50] Binance: $110,747.22 | MEXC: $110,744.38 | Bitget: $110,738.25 | 成功平台: 3/3
已收集: 437 筆，剩餘時間: 27分46秒，等待60.0秒...
[2025-09-06T18:48:52] Binance: $110,738.11 | MEXC: $110,735.39 | Bitget: $110,734.47 | 成功平台: 3/3
已收集: 438 筆，剩餘時間: 26分43秒，等待60.0秒...
[2025-09-06T18:49:55] Binance: $110,720.00 | MEXC: $110,720.97 | Bitget: $110,716.00 | 成功平台: 3/3
已收集: 439 筆，剩餘時間: 25分41秒，等待60.0秒...
[2025-09-06T18:50:57] Binance: $110,730.00 | MEXC: $110,727.30 | Bitget: $11